In [1]:
from multiprocessing.dummy import freeze_support
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['WANDB_NOTEBOOK_NAME'] = 'pytorch_stats_own_data.ipynb'
os.environ['WANDB_API_KEY'] = os.getenv('WANDB_API_KEY')


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

from darts import TimeSeries
from darts.models import TransformerModel
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller, FittableDataTransformer

from darts.metrics import mape, r2_score, rmse

from darts import TimeSeries

import helper
import glob

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(project="Digital-Energy")


AVAILABLE_GPUS = torch.cuda.device_count()
AVAILABLE_CPUS = os.cpu_count()

print(f"Available GPUs: {AVAILABLE_GPUS}")
print(f"Available CPUs: {AVAILABLE_CPUS}")

`fused_weight_gradient_mlp_cuda` module not found. gradient accumulation fusion with weight gradient computation disabled.
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find pytorch_stats_own_data.ipynb.
wandb: Currently logged in as: timmermansjoy (use `wandb login --relogin` to force relogin)


Available GPUs: 2
Available CPUs: 32


In [3]:
model = TransformerModel.load_from_checkpoint(work_dir="../..//Models", model_name="transformer")

[2022-05-02 14:14:12,871] INFO | darts.models.forecasting.torch_forecasting_model | loading best-epoch=4-val_loss=0.02.ckpt
[2022-05-02 14:14:12,871] INFO | darts.models.forecasting.torch_forecasting_model | loading best-epoch=4-val_loss=0.02.ckpt
INFO:darts.models.forecasting.torch_forecasting_model:loading best-epoch=4-val_loss=0.02.ckpt


In [7]:
for i in range(600,605):

    df = pd.read_csv(f'../../Data/london_clean/cleaned_household_MAC000{i}.csv')
    df["DateTime"] = pd.to_datetime(df['DateTime'])
    series = TimeSeries.from_dataframe(df, value_cols=['KWHhh'], time_col="DateTime", fill_missing_dates=True)
    series = series[-150:]


    pred_series = model.historical_forecasts(
        series,
        forecast_horizon=1,
        stride=1,
        retrain=False,
    )

    print(f"rmse: {rmse(series, pred_series)}.")
    print(f"R2 score: {r2_score(series, pred_series)}.")

    helper.display_forecast(pred_series, series, "1 day", save=True, fig_name=f"household_MAC000{i}", model_name=f"{MODEL_NAME}", fig_size=(20,10))

[2022-05-02 14:15:17,669] ERROR | main_logger | ValueError: All input series must have length >= `input_chunk_length` (125).
[2022-05-02 14:15:17,669] ERROR | main_logger | ValueError: All input series must have length >= `input_chunk_length` (125).
[2022-05-02 14:15:17,669] ERROR | main_logger | ValueError: All input series must have length >= `input_chunk_length` (125).
[2022-05-02 14:15:17,669] ERROR | main_logger | ValueError: All input series must have length >= `input_chunk_length` (125).
[2022-05-02 14:15:17,669] ERROR | main_logger | ValueError: All input series must have length >= `input_chunk_length` (125).
ERROR:main_logger:ValueError: All input series must have length >= `input_chunk_length` (125).


ValueError: All input series must have length >= `input_chunk_length` (125).